In [2]:
from soam.workflow import TimeSeriesExtractor
from muttlib.dbconn import BigQueryClient
import pandas as pd

In [57]:
db = "COSTS"
table =  "LK_GOOGLE_TOOLS_COST"
project = "meli-marketing"
bqcli = BigQueryClient(auth_file="meli-roioptimizer/pomopt/configs/bigquery_secret.json", db=db, table=table, project= project, auth=None)
tsstep = TimeSeriesExtractor(bqcli, bqcli.table_id)

In [58]:
bqcli.table_id

'meli-marketing.COSTS.LK_GOOGLE_TOOLS_COST'

In [59]:
columns = [
    "tim_day",
    "sit_site_id",
    "account_id",
    "account_name",
    "tool_id",
    "campaign_id",
    "campaign_name",
    "impressions",
    "clicks",
    "cost_lc",
    "cost_usd",
    "budget_amount",
    "budget_id",
]
dimensions = ["#account_id"]
dimensions_values = [3086755263]
extra_where_conditions =[
    "campaign_id IS NOT NULL", 
    "sit_site_id = 'MLM' "
]

build_query_kwargs = {
    "columns": columns,
    "dimensions": dimensions, 
    "dimensions_values": dimensions_values,
    "timestamp_col": "tim_day",
    "start_date": "2020-07-10",
    "extra_where_conditions": extra_where_conditions
}

In [60]:
query = tsstep.build_query(**build_query_kwargs)
print(query[0])


          SELECT tim_day, sit_site_id, account_id, account_name, tool_id, campaign_id, campaign_name, impressions, clicks, cost_lc, cost_usd, budget_amount, budget_id
          FROM meli-marketing.COSTS.LK_GOOGLE_TOOLS_COST
          
          
          WHERE account_id = 3086755263 AND tim_day >= '2020-07-10' AND campaign_id IS NOT NULL AND sit_site_id = 'MLM' 
          
          
          
          
        


### Cache query data

In [6]:
df = tsstep.run(build_query_kwargs)
df.to_csv("test_meli_camps_all.csv")

In [4]:
df = pd.read_csv("/home/fabian/Documents/Mutt/meli/test_meli_camps_all.csv")
df.head()

,Unnamed: 0,tim_day,sit_site_id,account_id,account_name,tool_id,campaign_id,campaign_name,impressions,clicks,cost_lc,cost_usd,budget_amount,budget_id
0,0,2020-08-23,MLM,3086755263,"MLM_ML_G_AO_SEARCH_MLM1000-Electrónica, Audio ...",6181903,104048113,"MLM_ML_G_AO_MLM1000-Electronica, Audio y Video...",3156,402,296.57,13.4585,6000.0,6.942559e+09
1,1,2020-08-21,MLM,3086755263,"MLM_ML_G_AO_SEARCH_MLM1000-Electrónica, Audio ...",6181903,104048113,"MLM_ML_G_AO_MLM1000-Electronica, Audio y Video...",2794,378,313.65,14.2215,6000.0,6.942559e+09
2,2,2020-07-14,MLM,3086755263,"MLM_ML_G_AO_SEARCH_MLM1000-Electrónica, Audio ...",6181903,104048113,"MLM_ML_G_AO_MLM1000-Electronica, Audio y Video...",3579,537,535.50,23.7661,NaN,NaN
3,3,2020-07-31,MLM,3086755263,"MLM_ML_G_AO_SEARCH_MLM1000-Electrónica, Audio ...",6181903,104048113,"MLM_ML_G_AO_MLM1000-Electronica, Audio y Video...",4040,620,581.30,26.4245,6000.0,6.942559e+09
4,4,2020-07-21,MLM,3086755263,"MLM_ML_G_AO_SEARCH_MLM1000-Electrónica, Audio ...",6181903,104048113,"MLM_ML_G_AO_MLM1000-Electronica, Audio y Video...",4472,750,813.81,36.1830,8000.0,6.942559e+09


# SoaM XD

In [3]:
from soam.models.prophet import SkProphet
from soam.workflow import Forecaster, Slicer, Anomaly, MergeConcat
from soam.core import SoamFlow
from prefect import task
import pandas as pd

In [10]:
proph = SkProphet(date_column="tim_day", yhat_only=False)


forecasting = Forecaster(proph, ds_col="tim_day", keep_cols="campaign_id", drop_after=True)
detecting = Anomaly(ds_col="tim_day", value_cols=["yhat_lower", "yhat_upper"],keep_cols="campaign_id")
slicing_anomalies = Slicer(dimensions="campaign_id", metrics=["clicks", "cost_lc", "impressions"], ds_col="tim_day")
merge_con = MergeConcat(keys=["tim_day", "campaign_id"])

@task
def read_data():
    return pd.read_csv("test_meli_camps_all.csv")

@task
def save_df(df):
    df.to_csv("test_meli_outliear.csv")

In [13]:
with SoamFlow(name="meli_metrics_ouliears") as fs:
    # df = tsstep(build_query_kwargs)
    df = read_data()
    dfs_sliced = slicing_anomalies(df)
    # dfs_sliced = dfs_sliced[:12]
    forecasted = forecasting.map(dfs_sliced)
    anomalies = detecting.map(forecasted)
    final_df = merge_con(anomalies)
    save_df(final_df)

In [14]:
fs.run()

refect.TaskRunner | Task 'Anomaly[79]': Starting task run...
INFO:prefect.TaskRunner:Task 'Anomaly[79]': Starting task run...
[2020-10-30 14:52:49] INFO - prefect.TaskRunner | Task 'Anomaly[79]': Finished task run for task with final state: 'Success'
INFO:prefect.TaskRunner:Task 'Anomaly[79]': Finished task run for task with final state: 'Success'
[2020-10-30 14:52:49] INFO - prefect.TaskRunner | Task 'Anomaly[80]': Starting task run...
INFO:prefect.TaskRunner:Task 'Anomaly[80]': Starting task run...
[2020-10-30 14:52:49] INFO - prefect.TaskRunner | Task 'Anomaly[80]': Finished task run for task with final state: 'Success'
INFO:prefect.TaskRunner:Task 'Anomaly[80]': Finished task run for task with final state: 'Success'
[2020-10-30 14:52:49] INFO - prefect.TaskRunner | Task 'Anomaly[81]': Starting task run...
INFO:prefect.TaskRunner:Task 'Anomaly[81]': Starting task run...
[2020-10-30 14:52:49] INFO - prefect.TaskRunner | Task 'Anomaly[81]': Finished task run for task with final state:

<Success: "All reference tasks succeeded.">